In [1]:
!pip install transformers langchain openai accelerate bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

# Retrieve the Hugging Face API token securely
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# Authenticate with Hugging Face
login(token=hf_token)

print("✅ Successfully logged into Hugging Face!")

✅ Successfully logged into Hugging Face!


In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# 4-bit quantization config
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, max_length=512)

# Load model with 4-bit quantization & move to GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    token=hf_token,
    device_map="auto",  # Automatically assigns layers to GPU
    torch_dtype=torch.float32  # Use FP16 for efficiency
)

print("✅ Model loaded with 4-bit quantization on GPU!")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Model loaded with 4-bit quantization on GPU!


In [4]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [5]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Load Hugging Face pipeline
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=768)

# Wrap in LangChain LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Create a PromptTemplate
template = """
Convert this natural language question to SQL:
Schema: {schema}
Question: {question}
SQL:
"""

prompt = PromptTemplate(input_variables=["schema", "question"], template=template)

# Create LLM Chain
llm_chain = LLMChain(llm=llm, prompt=prompt)


Device set to use cuda:0
<ipython-input-5-125fad858ba4>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)
<ipython-input-5-125fad858ba4>:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
schema = '''CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price DECIMAL(10,2),
    minimum_nights INT,
    maximum_nights INT,
    review_scores_rating INT,
    review_scores_accuracy DECIMAL(3,1),
    review_scores_cleanliness DECIMAL(3,1),
    review_scores_checkin DECIMAL(3,1),
    review_scores_communication DECIMAL(3,1),
    review_scores_location DECIMAL(3,1),
    review_scores_value DECIMAL(3,1),
    instant_bookable BOOLEAN
);
CREATE TABLE IF NOT EXISTS reviews (
    listing_id INT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES airbnb_listings(listing_id) ON DELETE CASCADE
);'''  
question = "What is the total number of reviews for each listing?"

sql_query = llm_chain.invoke({"schema": schema, "question": question})
print(sql_query["text"])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Convert this natural language question to SQL:
Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenit

In [7]:
from langchain import PromptTemplate

zero_shot_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="Convert NL question to SQL:\nSchema: {schema}\nQuestion: {question}\nFinal SQL Query:"
)

few_shot_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
Convert NL question to SQL:
Schema: {schema}

Example 1:
Q: How many users exist?
SQL: SELECT COUNT(*) FROM Users;

Example 2:
Q: Show reviews with a rating above 4.
SQL: SELECT * FROM Reviews WHERE rating > 4;

Now answer:
Q: {question}
Final SQL Query:
"""
)

cot_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
Convert NL to SQL step-by-step:
1. Identify tables.
2. Determine conditions.
3. Construct the SQL query.

Schema: {schema}
Question: {question}
Steps:

Final SQL Query:
"""
)

tot_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are an SQL expert. Break down the problem step by step before generating the final SQL query.

**Schema:** {schema}

**User Query:** "{question}"

**Thought Process:**
1. Identify the relevant tables and columns.
2. Determine the necessary joins and filtering conditions.
3. Select the appropriate SQL operations (aggregation, sorting).
4. Construct the SQL query.

Final SQL Query:
"""
)

got_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are an SQL expert. Consider multiple paths to derive the best SQL query.

**Schema:** {schema}

**User Query:** "{question}"

**Graph of Thought Reasoning:**
- **Path 1:** Analyze the intent of the question.
- **Path 2:** Identify relevant tables, columns, and joins.
- **Path 3:** Explore multiple query structures.
- **Path 4:** Merge the best paths for optimal SQL.

Final SQL Query:
"""
)

react_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are an SQL expert. Follow these steps to generate and validate an SQL query:

### Step 1: Understand the Schema
- Analyze the provided schema to identify relevant tables, columns, and relationships.
- Schema:
{schema}

### Step 2: Understand the Question
- Carefully read the question to determine what data is being requested.
- Question:
{question}

### Step 3: Plan the Query
- Identify the tables and columns needed to answer the question.
- Determine any joins, filters, or aggregations required.

### Step 4: Generate the SQL Query
- Write an SQL query that retrieves the requested data.
- Ensure the query is syntactically correct and logically aligns with the question.

### Step 5: Validate the Query
- Double-check the query for correctness and efficiency.

Final SQL Query:
"""
)

self_consistency_template = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
You are an SQL expert. Generate multiple candidate SQL queries for the given question, then select the best one.

Schema:
{schema}

Question:
{question}

Candidate Queries:
{{candidate_sql_1}}
{{candidate_sql_2}}
{{candidate_sql_3}}

Validation and Selection:
- Compare the candidates for correctness and efficiency.
- Select the best query.

Final SQL Query:
"""
)


In [8]:
def generate_sql(query, schema, technique="zero_shot"):
    prompt_templates = {
        "zero_shot": zero_shot_template,
        "few_shot": few_shot_template,
        "cot": cot_template,
        "tot": tot_template,
        "got": got_template,
        "react": react_template,
        "self_consistency": self_consistency_template
    }

    if technique not in prompt_templates:
        raise ValueError("Invalid prompting technique. Choose from: zero_shot, few_shot, cot, tot, got, react.")

    # Create prompt using selected technique
    prompt = prompt_templates[technique]

    # Define LangChain chain
    chain = LLMChain(llm=llm, prompt=prompt)

    # Generate SQL
    result = chain.invoke({"schema": schema, "question": query})

    return result["text"]


In [9]:
query = "Find the top 5 Airbnb listings in a given city with the highest average review score, considering only listings with at least 10 reviews."

# Select prompting technique
technique = "got"  # Change this to "tot", "got", etc.

# Generate SQL query
sql_query = generate_sql(query, schema, technique)
print(sql_query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Consider multiple paths to derive the best SQL query.

**Schema:** CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodate

# sql-loading

In [10]:
import pandas as pd
df = pd.read_csv("/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings.csv", encoding="ISO-8859-1")
df.replace({"t":1, "f":0}, inplace=True)
df = df.replace({pd.NA: None, float("nan"): None})
df = df.iloc[:10000, :]

<ipython-input-10-a6037faf6a31>:2: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/airbnb-listings-reviews/Airbnb Data/Listings.csv", encoding="ISO-8859-1")
<ipython-input-10-a6037faf6a31>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"t":1, "f":0}, inplace=True)


In [ ]:
!apt install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-lite23
  liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils mysql-client-8.0 mysql-client-core-8.0
  mysql-server-8.0 mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-l

In [ ]:
!sudo service mysql restart

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector

# Database Connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
)
cursor = conn.cursor()
query = "create database llm"
cursor.execute(query)

# Commit and Close Connection
conn.commit()
cursor.close()
conn.close()

In [ ]:
import mysql.connector

# Database Connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="llm"
)
cursor = conn.cursor()

# SQL Query to Create Table
create_table_query = """
CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price DECIMAL(10,2),
    minimum_nights INT,
    maximum_nights INT,
    review_scores_rating INT,
    review_scores_accuracy DECIMAL(3,1),
    review_scores_cleanliness DECIMAL(3,1),
    review_scores_checkin DECIMAL(3,1),
    review_scores_communication DECIMAL(3,1),
    review_scores_location DECIMAL(3,1),
    review_scores_value DECIMAL(3,1),
    instant_bookable BOOLEAN
);
"""

# Execute Query
cursor.execute(create_table_query)

# Commit and Close Connection
conn.commit()
cursor.close()
conn.close()

print("Table 'airbnb_listings' created successfully!")

In [ ]:
df = df.dropna(subset=["name"])

In [ ]:
df["name"] = df["name"].apply(lambda x: x if len(x) < 255 else x[:255])

In [ ]:
import mysql.connector
import pandas as pd

# Reconnect to the database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="llm"
)
cursor = conn.cursor()

# SQL query to insert data
insert_query = """
INSERT INTO airbnb_listings (
    listing_id, name, host_id, host_since, host_location, host_response_time, 
    host_response_rate, host_acceptance_rate, host_is_superhost, host_total_listings_count, 
    host_has_profile_pic, host_identity_verified, neighbourhood, district, city, 
    latitude, longitude, property_type, room_type, accommodates, bedrooms, amenities, 
    price, minimum_nights, maximum_nights, review_scores_rating, review_scores_accuracy, 
    review_scores_cleanliness, review_scores_checkin, review_scores_communication, 
    review_scores_location, review_scores_value, instant_bookable
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Iterate over DataFrame and insert data
for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")


In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

# Database connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="llm"
)
cursor = conn.cursor()

# SQL query to create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS reviews (
    listing_id INT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL
);
"""

# Execute query
cursor.execute(create_table_query)
conn.commit()
print("Table 'reviews' created successfully.")

# Load CSV data
conn.close()

In [ ]:

csv_file = "/kaggle/input/airbnb-listings-reviews/Airbnb Data/Reviews.csv"  # Update this with the actual file path
df = pd.read_csv(csv_file)

# Drop rows with missing essential fields

# Convert date column to MySQL format
df["date"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')

In [ ]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="llm"
)
cursor = conn.cursor()
# Prepare SQL query for insertion
insert_query = """
INSERT INTO reviews (listing_id, review_id, date, reviewer_id) 
VALUES (%s, %s, %s, %s)
"""

# Insert data
for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

# evaluation

In [11]:
query1 = "Find the total number of listings in each city"
query2 = "Find the oldest host in each city"

In [12]:
print(generate_sql(query1, schema, "zero_shot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Convert NL question to SQL:
Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price 

In [13]:
print(generate_sql(query2, schema, "zero_shot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Convert NL question to SQL:
Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price 

In [14]:
print(generate_sql(query1, schema, "few_shot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Convert NL question to SQL:
Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price

In [26]:
print(generate_sql(query2, schema, "few_shot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Convert NL question to SQL:
Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodates INT,
    bedrooms INT,
    amenities TEXT,
    price

In [16]:
print(generate_sql(query1, schema, "cot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Convert NL to SQL step-by-step:
1. Identify tables.
2. Determine conditions.
3. Construct the SQL query.

Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCH

In [17]:
print(generate_sql(query2, schema, "cot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Convert NL to SQL step-by-step:
1. Identify tables.
2. Determine conditions.
3. Construct the SQL query.

Schema: CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCH

In [18]:
print(generate_sql(query1, schema, "tot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Break down the problem step by step before generating the final SQL query.

**Schema:** CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(

In [19]:
print(generate_sql(query2, schema, "tot"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Break down the problem step by step before generating the final SQL query.

**Schema:** CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(

In [20]:
print(generate_sql(query1, schema, "got"))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Consider multiple paths to derive the best SQL query.

**Schema:** CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodate

In [21]:
print(generate_sql(query2, schema, "got"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Consider multiple paths to derive the best SQL query.

**Schema:** CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_type VARCHAR(100),
    accommodate

In [22]:
print(generate_sql(query1, schema, "react"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Follow these steps to generate and validate an SQL query:

### Step 1: Understand the Schema
- Analyze the provided schema to identify relevant tables, columns, and relationships.
- Schema:
CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latit

In [23]:
print(generate_sql(query2, schema, "react"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Follow these steps to generate and validate an SQL query:

### Step 1: Understand the Schema
- Analyze the provided schema to identify relevant tables, columns, and relationships.
- Schema:
CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latit

In [24]:
print(generate_sql(query1, schema, "self_consistency"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Generate multiple candidate SQL queries for the given question, then select the best one.

Schema:
CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_ty

In [25]:
print(generate_sql(query2, schema, "self_consistency"))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an SQL expert. Generate multiple candidate SQL queries for the given question, then select the best one.

Schema:
CREATE TABLE reviews (
    listing_id BIGINT NOT NULL,
    review_id BIGINT PRIMARY KEY,
    date DATE NOT NULL,
    reviewer_id BIGINT NOT NULL,
    FOREIGN KEY (listing_id) REFERENCES listings(listing_id),
    FOREIGN KEY (reviewer_id) REFERENCES reviewers(reviewer_id),
    CREATE TABLE IF NOT EXISTS airbnb_listings (
    listing_id INT PRIMARY KEY,
    name VARCHAR(255),
    host_id INT,
    host_since DATE,
    host_location VARCHAR(255),
    host_response_time VARCHAR(50),
    host_response_rate DECIMAL(5,2),
    host_acceptance_rate DECIMAL(5,2),
    host_is_superhost BOOLEAN,
    host_total_listings_count INT,
    host_has_profile_pic BOOLEAN,
    host_identity_verified BOOLEAN,
    neighbourhood VARCHAR(255),
    district VARCHAR(255),
    city VARCHAR(255),
    latitude DECIMAL(10,6),
    longitude DECIMAL(10,6),
    property_type VARCHAR(100),
    room_ty